In [382]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

class StopExecution(Exception):
    def _render_traceback_(self):
        return []



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [383]:
import sys
sys.path.append("..")
import pickle as pkl
import hist
from analyzer.datasets import SampleManager
from analyzer.core import AnalysisResult
import math
import torch
import gpytorch
from torch.masked import masked_tensor, as_masked_tensor
import numpy as np

In [384]:
import matplotlib.pyplot as plt
from analyzer.plotting.plots_1d import drawAs1DHist
from analyzer.plotting.plots_2d import drawAs2DHist, addTitles2D
from analyzer.plotting.plottables import PlotObject
from analyzer.plotting.mplstyles import loadStyles
import analyzer.plotting as plot
loadStyles()

In [385]:
from fitting import regression
from fitting import plot_tools as fplt


In [386]:
res = AnalysisResult.fromFile("../results/data_control.pkl")

In [387]:
sample_manager = SampleManager()
sample_manager.loadSamplesFromDirectory("../datasets")

In [388]:
res.results["CR0b_Data2018"].histograms["h_njet"] 

Hist(
  StrCategory(['CR0b_Data2018'], growth=True, name='dataset', label='Primary dataset'),
  Regular(10, 0, 10, name='nj', label='$n_{j}$'),
  storage=Weight()) # Sum: WeightedSum(value=1.40454e+07, variance=1.40454e+07)

In [389]:
hists = res.getMergedHistograms(sample_manager)

In [390]:
bkg_name="CR0b_Data2018"
complete_hist = hists["ratio_m14_vs_m24"]
narrowed = complete_hist
#orig =  complete_hist[...,hist.loc(1100):hist.loc(2500),hist.loc(400):hist.loc(2000)]
#narrowed =  complete_hist[...,hist.loc(0):hist.loc(3000),hist.loc(0):hist.loc(1)]
narrowed =  complete_hist[...,hist.loc(1150):hist.loc(3000),hist.loc(0.4):hist.loc(1)]
#orig = orig / orig.values().max()
#orig =  complete_hist[...,hist.loc(1150):hist.loc(3000),hist.loc(400):hist.loc(3000)]#orig =  complete_hist[...,hist.loc(0):hist.loc(3000),hist.loc(0):hist.loc(3000)]
#narrowed = orig
#narrowed = orig[...,::hist.rebin(15),::hist.rebin(12)]
narrowed = narrowed[...,::hist.rebin(1),::hist.rebin(1)]

qcd_hist = narrowed[bkg_name,...]
#orig_qcd_hist = orig[bkg_name,...]

#sig_hist = narrowed["signal_312_1500_900",...]

qcd_hist = narrowed[bkg_name,...] 


In [391]:
x = qcd_hist * 1 / value_scale.item()
print(x.variances().max())
print(train.variances.max())

2.384642899725766e-05


AttributeError: 'DataValues' object has no attribute 'variances'

In [ ]:
torch.set_default_dtype(torch.float64)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
def addColorbar(ax, vals):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(vals, cax=cax)
    cax.get_yaxis().set_offset_position("left")
    ax.cax = cax

In [ ]:
fig,ax= plt.subplots(1,2,figsize=(10,3))
_ = drawAs2DHist(ax[0], PlotObject.fromHist(qcd_hist))
#_ = drawAs2DHist(ax[1], PlotObject.fromHist(sig_hist))

In [ ]:
window = [(1300, 1550), (0.46, 0.6)]

train_data = regression.makeRegressionData(qcd_hist[hist.rebin(1),hist.rebin(1)], window, exclude_less=1)
test_data = regression.makeRegressionData(qcd_hist[hist.rebin(1),hist.rebin(1)], None, exclude_less=1)

train_transform = regression.getNormalizationTransform(train_data)
test_transform = regression.getNormalizationTransform(test_data)

normalized_train_data = train_transform.transform(train_data)
normalized_test_data = test_transform.transform(test_data)

print(train_transform)
print(test_transform)
print(train_data.X[:10])
print(test_data.X[:10])

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10),layout="tight")
fplt.simpleGrid(ax[0][0], train_data.E, train_data.X, train_data.Y)
fplt.simpleGrid(ax[0][1], test_data.E, test_data.X, test_data.Y)
fplt.simpleGrid(ax[1][0], normalized_train_data.E, normalized_train_data.X, normalized_train_data.Y)
fplt.simpleGrid(ax[1][1], normalized_test_data.E, normalized_test_data.X, normalized_test_data.Y)

In [ ]:
class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self, odim=2):
        super().__init__()
        self.add_module('linear1', torch.nn.Linear(2, 1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 500))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(500, 50))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(50, odim))


In [ ]:
class NNModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood,kernel=None,odim=2):
            super().__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            
            self.covar_module = kernel or gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2))
            feature_extractor = LargeFeatureExtractor(odim=odim)
            self.feature_extractor = feature_extractor

            # This module will scale the NN features so that they're nice values
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor(x)
            projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
from fitting import regression
from gpytorch.kernels import ScaleKernel as SK
from gpytorch.kernels import RBFKernel as RBF

import fitting.models as models
#importlib.reload(models)
#importlib.reload(regression)


if torch.cuda.is_available():   
    print("Loading train to gpu")
    train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)
    print("Loaded train to gpu")
else:
    train = normalized_train_data
    


# model,likelihood = regression.createModel(
#     train_data, 
#     kernel=SK(
#         gpytorch.kernels.GridInterpolationKernel(
#             gpytorch.kernels.RBFKernel(ard_num_dims=2), grid_size=grid_size, num_dims=2)
#     ))
#model,likelihood = regression.createModel(train_data, 
#kernel=SK(
#                                              gpytorch.kernels.RBFKernel(ard_num_dims=2) + 
#                                              gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2)
#                                          ))
#model,likelihood = regression.createModel(train_data, kernel=SK(gpytorch.kernels.MaternKernel(ard_num_dims=2)))
#model,likelihood = regression.createModel(train_data, kernel=SK(gpytorch.kernels.PiecewisePolynomialKernel(ard_num_dims=2)))
#model,likelihood = regression.createModel(train_data, model_maker= ExactProjGPModel )#model,likelihood = regression.createModel(train, kernel=SK(models.MatrixRBF()))
#model,likelihood = regression.createModel(train_data, kernel=SK(models.PeakedRBF(ard_num_dims=2)))
#known_mean = models.HeterogenousConstantMean(train_data.Y)

smk = gpytorch.kernels.SpectralMixtureKernel(ard_num_dims=2, num_mixtures=4)
gsmk=models.GeneralSpectralMixture(num_mixtures=1, ard_num_dims=2)
smk.initialize_from_data(train.X, train.Y)
#gsmk.initialize_from_data(train.inputs, train.outputs)

grbf = models.GeneralRBF(ard_num_dims=2)
lk = gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2)
rbf = gpytorch.kernels.RBFKernel(ard_num_dims=2)

#model,likelihood = regression.createModel(train, kernel=gpytorch.kernels.SpectralMixtureKernel(ard_num_dims=2, num_mixtures=2))
#model,likelihood = regression.createModel(train, kernel=smk)
#model,likelihood = regression.createModel(train, model_maker=NNModel, kernel=rbf, odim=2)
#model,likelihood = regression.createModel(train, kernel=induc_grbf, mean=None, learn_noise=False)

#model,likelihood = regression.createModel(train, kernel=gpytorch.kernels.SpectralDeltaKernel(num_dims=2, ard_num_dims=2))#model,likelihood = regression.createModel(train, kernel=SK(gpytorch.kernels.RQKernel(ard_num_dims=2)))
#model,likelihood = regression.createModel(train, kernel=SK(models.GeneralRBF(ard_num_dims=2)))
#model,likelihood = regression.createModel(train, kernel=SK(models.GeneralRQ(ard_num_dims=2)))
#model,likelihood = regression.createModel(train, kernel=models.GeneralSpectralMixture(num_mixtures=1, ard_num_dims=2))
#model,likelihood = regression.createModel(train, kernel=grbf)
print(model)
def create4Part(k,*args,**kwargs):
    return ( k(*args,**kwargs,ard_num_dims=2,active_dims=(0,0))
          *k(*args,**kwargs,ard_num_dims=2,active_dims=(1,0))            
          *k(*args,**kwargs,ard_num_dims=2,active_dims=(0,1))
          *k(*args,**kwargs,ard_num_dims=2,active_dims=(1,1))
           )
#model.covar_module.kernels[1].initialize_from_data(train.inputs, train.outputs)
#model.covar_module.base_kernel.initialize_from_data(train.inputs, train.outputs)
#model.covar_module.initialize_from_data_empspect(train.inputs, train.outputs)



likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(
        noise=train.V,
        learn_additional_noise=False,
    )
rbf4 = create4Part(gpytorch.kernels.RBFKernel)

#model,likelihood = regression.createModel(train, kernel=rbf4)


class InducingPointModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, inducing):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.base_covar_module = rbf4
        self.covar_module = gpytorch.kernels.InducingPointKernel(self.base_covar_module, inducing_points=inducing.clone(), likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
model = InducingPointModel(train.X,train.Y, likelihood, train.X[::2])
if torch.cuda.is_available():   
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    #train = regression.DataValues(train.inputs.cuda(), train.outputs.cuda(), train.variances.cuda(), None)
    print("Loaded model to gpu")
    
model,likelihood = regression.optimizeHyperparams(model,likelihood, train, bar=False, iterations=400, lr=0.1)

for n,p in model.named_parameters():    
    print(f"{n:45} {p.data}")

In [ ]:
print(model.covar_module.base_kernel.getMatrix())
print(model.covar_module.base_kernel.rot)
print(model.covar_module.base_kernel.lengthscale)
#print(model.covar_module.getMatrix())
#print(model.covar_module.rot)
#print(model.covar_module.lengthscale)



#print(model.covar_module.base_kernel.alpha)

In [ ]:
import linear_operator
#with linear_operator.settings.max_cg_iterations(10000):
#    pred = regression.getPrediction(model, likelihood, test_data)
pred = regression.getPrediction(model, likelihood, normalized_test_data)


from mpl_toolkits.axes_grid1 import make_axes_locatable
import itertools as it
from fitting.plot_tools import simpleGrid
#window = [(1300,1550),(400,600)]
print(test_transform)
pred = test_transform.iTransform(
    regression.DataValues(normalized_test_data.X, pred.mean, pred.variance, normalized_test_data.E))


if window:
    print(regression.getChi2Blinded(pred.X, pred.Y, test_data.Y, test_data.V, window))

all_pulls = (pred.Y - test_data.Y) / torch.sqrt(test_data.V)
all_x2 = (pred.Y - test_data.Y)**2 / test_data.V
x2 = torch.sum(all_x2)

def addColorbar(ax, vals):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(vals, cax=cax)
    cax.get_yaxis().set_offset_position("left")
    ax.cax = cax

fig, ax=plt.subplots(layout="tight")
f = simpleGrid(ax, train_data.E, train_data.X, train_data.Y)
ax.set_title("Masked Inputs (Training)")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
#fig.savefig(figpath / "training_points.pdf")

fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, pred.E, pred.X, pred.Y)
ax.set_title("GPR Mean Prediction")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
#fig.savefig(figpath / "gpr_mean.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, test_data.Y)
ax.set_title("Observed Outputs")
#fig.savefig(figpath / "observed_outputs.pdf")



#drawAs2DHist(ax[1,1], PlotObject.fromHist(sig_hist))
#addTitles2D(ax[1,1], PlotObject.fromHist(sig_hist))
#ax[1,1].set_title("Signal MC")

fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, test_data.V)
ax.set_title("Observed Variances")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))

#fig.savefig(figpath / "observed_variances.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, pred.E, pred.X, pred.V)
ax.set_title("Pred Variances")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))

#fig.savefig(figpath / "predicted_variances.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, (test_data.Y - pred.Y)/ torch.sqrt(test_data.V))
f.set_clim(-5,5)
ax.set_title("Pulls")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
ax.cax.set_ylabel(r"$\frac{N_{obs}-N_{pred}}{\sigma_{p}}$")
#fig.savefig(figpath / "pulls.pdf")



import uhi
fig, ax=plt.subplots(figsize=(10,10), layout="tight")
p = PlotObject.fromHist(uhi.numpy_plottable.ensure_plottable_histogram(np.histogram(all_pulls[torch.abs(all_pulls) < np.inf], bins=20)))
drawAs1DHist(ax, p, yerr=False)
ax.set_xlabel(r"$\frac{N_{obs}-N_{pred}}{\sigma_{o}}$")
ax.set_ylabel("Count")
#fig.savefig(figpath / "pulls_hist.pdf")

In [ ]:
import fitting.plot_tools as fpt
import fitting
import analyzer.plotting as plotting
from pathlib import Path
import importlib

importlib.reload(fpt)
dim = 1

pred_mean,_ = fitting.regression.pointsToGrid(test_data.X, pred.Y, test_data.E)
pred_var,_ = fitting.regression.pointsToGrid(test_data.X, pred.V, test_data.E)
obs_vals,_ = fitting.regression.pointsToGrid(test_data.X, test_data.Y, test_data.E)
obs_vars, filled = fitting.regression.pointsToGrid(test_data.X, test_data.V, test_data.E)

#(figpath /"slices" / f"along_{dim}").mkdir(parents=True, exist_ok=True)
for val, f, ax in fpt.createSlices(
    pred_mean.hist,
    pred_var.hist,
    obs_vals.hist,
    obs_vars.hist,
    test_data.E,
    filled,
    observed_title="CRData", window_2d=window, dim=dim):
    plotting.addTitles1D(ax, plotting.PlotObject.fromHist(qcd_hist[:,sum]))
    #f.savefig(figpath /"slices" / f"along_{dim}" /  (f"slice_{round(float(val),3)}".replace(".","p") + ".pdf"))
    #plt.close(f)
    
    